# Adding Memory to a Chain (Part 1): Implementing the Setup

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
%load_ext dotenv
%dotenv

In [17]:
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from operator import itemgetter

In [3]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [4]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/opt/anaconda3/envs/langchain_env_benji/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [5]:
chain = prompt_template | chat | StrOutputParser()

In [6]:
chain.invoke({'message_log':'''The AI provides an interesting fact about octopuses, stating that they have three hearts. 
Two hearts pump blood to the gills, while the third heart pumps it to the rest of the body.''', 
              'question':"Can you elaborate a bit more on this fact?"})

"Absolutely! Octopuses are fascinating creatures with a unique cardiovascular system. They have three hearts, each serving a specific purpose. Two of these hearts, known as branchial hearts, are located near each of the octopus's two gills. These branchial hearts pump blood to the gills, where it gets oxygenated.\n\nThe third heart, known as the systemic heart, pumps the oxygenated blood from the gills to the rest of the body. This heart actually stops beating when the oct"

In [7]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')

/var/folders/l0/l8bzvy196wn95cjygzkqtsfw0000gn/T/ipykernel_98022/3865083148.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')


In [9]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [16]:
RunnablePassthrough.assign(
    message_log = chat_memory.load_memory_variables
).invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_log': {'message_log': ''}}

In [ ]:
RunnablePassthrough.assign(
    first_letter = lambda x: list(x['input'])[0],
    second_letter = lambda x: list(x['input'])[1]
).invoke({'input':"hi?"})

In [18]:
RunnablePassthrough.assign(
    message_log = RunnableLambda(chat_memory.load_memory_variables) | RunnableLambda(itemgetter('message_log'))
).invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_log': ''}